In [1]:
from kgdata.dbpedia.datasets.classes import *
from kgdata.misc.ntriples_parser import *
from kgdata.models.ont_class import *
import serde.textline

In [2]:
cfg = DBpediaDataDirCfg.init("/nas/ckgfs/users/binhvu/dbpedia/20230512")

In [4]:
props = ontology_dump().get_rdd().filter(is_prop).collect()

In [6]:
{x for p in props for x in p.props.get(str(RDFS.range), [])}

{rdflib.term.URIRef('http://dbpedia.org/datatype/engineConfiguration'),
 rdflib.term.URIRef('http://dbpedia.org/datatype/valvetrain'),
 rdflib.term.URIRef('http://dbpedia.org/ontology/Actor'),
 rdflib.term.URIRef('http://dbpedia.org/ontology/Agent'),
 rdflib.term.URIRef('http://dbpedia.org/ontology/Agglomeration'),
 rdflib.term.URIRef('http://dbpedia.org/ontology/Airport'),
 rdflib.term.URIRef('http://dbpedia.org/ontology/Altitude'),
 rdflib.term.URIRef('http://dbpedia.org/ontology/AnatomicalStructure'),
 rdflib.term.URIRef('http://dbpedia.org/ontology/Animal'),
 rdflib.term.URIRef('http://dbpedia.org/ontology/Annotation'),
 rdflib.term.URIRef('http://dbpedia.org/ontology/Architect'),
 rdflib.term.URIRef('http://dbpedia.org/ontology/Area'),
 rdflib.term.URIRef('http://dbpedia.org/ontology/Arena'),
 rdflib.term.URIRef('http://dbpedia.org/ontology/Artery'),
 rdflib.term.URIRef('http://dbpedia.org/ontology/Artist'),
 rdflib.term.URIRef('http://dbpedia.org/ontology/Astronaut'),
 rdflib.ter

In [7]:
ontology_dump().get_rdd().filter(is_prop).flatMap(lambda r: [(str(uri), r.id) for uri in r.props.get(str(RDFS.domain), [])]).take(1)

[('http://dbpedia.org/ontology/جگہ',
  'http://dbpedia.org/ontology/تاریخ_کا_معاہدہ')]

In [5]:
ontology_dump().get_rdd().filter(lambda x: 'http://www.w3.org/2002/07/owl#disjointWith' in x.props).take(1)

[RDFResource(id='http://dbpedia.org/ontology/Mammal', props={'http://www.w3.org/2000/01/rdf-schema#label': [rdflib.term.Literal('mamífero', lang='es'), rdflib.term.Literal('ssak', lang='pl'), rdflib.term.Literal('mamífero', lang='pt'), rdflib.term.Literal('zoogdier', lang='nl'), rdflib.term.Literal('mamach', lang='ga'), rdflib.term.Literal('mammal', lang='en'), rdflib.term.Literal('mammifère', lang='fr'), rdflib.term.Literal('θηλαστικό ζώο', lang='el'), rdflib.term.Literal('mammifero', lang='it'), rdflib.term.Literal('تھن والے جانور', lang='ur'), rdflib.term.Literal('pattedyr', lang='da'), rdflib.term.Literal('哺乳類', lang='ja'), rdflib.term.Literal('säugetier', lang='de')], 'http://www.w3.org/2000/01/rdf-schema#subClassOf': [rdflib.term.URIRef('http://dbpedia.org/ontology/Animal')], 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': [rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class')], 'http://www.w3.org/2002/07/owl#disjointWith': [rdflib.term.URIRef('http://dbpedia.org/ontology/

In [3]:
infile = list((cfg.ontology_dump / "step1").glob("*.gz"))[2]
lines = serde.textline.deser(infile)

In [4]:
lines[:10]

['<http://dbpedia.org/datatype/Area> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2000/01/rdf-schema#Datatype> .\n',
 '<http://dbpedia.org/datatype/Density> <http://www.w3.org/2000/01/rdf-schema#isDefinedBy> <http://dbpedia.org/ontology/> .\n',
 '<http://dbpedia.org/datatype/Energy> <http://www.w3.org/2002/07/owl#sameAs> <http://dbpedia.org/datatype/Energy> .\n',
 '<http://dbpedia.org/datatype/Force> <http://www.w3.org/2002/07/owl#sameAs> <http://dbpedia.org/datatype/Force> .\n',
 '<http://dbpedia.org/datatype/FuelEfficiency> <http://www.w3.org/2002/07/owl#sameAs> <http://dbpedia.org/datatype/FuelEfficiency> .\n',
 '<http://dbpedia.org/datatype/LinearMassDensity> <http://www.w3.org/2000/01/rdf-schema#label> "LinearMassDensity"@en .\n',
 '<http://dbpedia.org/datatype/PopulationDensity> <http://www.w3.org/2000/01/rdf-schema#label> "PopulationDensity"@en .\n',
 '<http://dbpedia.org/datatype/Pressure> <http://www.w3.org/2000/01/rdf-schema#label> "Pressure"@en .\n',


In [5]:
lit = ntriple_loads(lines[9])[2]

In [7]:
RDFTerm.from_dict(RDFTerm(lit).to_dict()).to_dict()

{'type': 'Literal', 'value': 'Time', 'datatype': None, 'language': 'en'}

In [52]:
from rdflib import RDFS

def is_class(resource) -> bool:
    return any(OWL.Class == v.term for v in resource.props.get(rdf_type, []))
    # return str(OWL.Class) in resource.props.get(rdf_type, [])

def to_class(resource, default_lang: str = 'en') -> OntologyClass:
    return OntologyClass(
        id=resource.id, 
        label=as_multilingual(resource.props[str(RDFS.label)]),
        description=as_multilingual(resource.props.get(str(RDFS.comment), [])),
        aliases=MultiLingualStringList({default_lang: []}, default_lang),
        parents=[str(term.term) for term in resource.props.get(str(RDFS.subClassOf), [])],
        properties=[],
        different_froms=[],
        equivalent_classes=[str(term.term) for term in resource.props.get(str(OWL.equivalentClass), [])],
        ancestors=set()
    )


def as_multilingual(terms: list[RDFTerm], default_lang: str = 'en'):
    if len(terms) == 0:
        return MultiLingualString({default_lang: ""}, default_lang)
        
    lang2value = {
        term.term.language: term.term.value
        for term in terms
    }
    assert default_lang in lang2value
    return MultiLingualString(lang2value, default_lang)

In [42]:
lst = (
    ontology_dump()
    .get_rdd()
    .filter(is_class)
    .filter(lambda x: x.id.endswith('City'))
    .take(5)
)

In [44]:
orjson.loads(lst[0].ser())

{'id': 'http://dbpedia.org/ontology/City',
 'props': {'http://www.w3.org/2000/01/rdf-schema#isDefinedBy': [{'type': 'URIRef',
    'value': 'http://dbpedia.org/ontology/'}],
  'http://www.w3.org/2000/01/rdf-schema#label': [{'type': 'Literal',
    'value': 'cathair',
    'datatype': None,
    'language': 'ga'},
   {'type': 'Literal', 'value': 'ville', 'datatype': None, 'language': 'fr'},
   {'type': 'Literal', 'value': 'Stadt', 'datatype': None, 'language': 'de'},
   {'type': 'Literal', 'value': 'cidade', 'datatype': None, 'language': 'gl'},
   {'type': 'Literal', 'value': 'πόλη', 'datatype': None, 'language': 'el'},
   {'type': 'Literal', 'value': 'cidade', 'datatype': None, 'language': 'pt'},
   {'type': 'Literal', 'value': 'شہر', 'datatype': None, 'language': 'ur'},
   {'type': 'Literal', 'value': 'città', 'datatype': None, 'language': 'it'},
   {'type': 'Literal', 'value': 'शहर', 'datatype': None, 'language': 'hi'},
   {'type': 'Literal', 'value': 'city', 'datatype': None, 'language'

In [53]:
to_class(lst[0])

OntologyClass(id='http://dbpedia.org/ontology/City', label='city', description='a relatively large and permanent settlement, particularly a large urban settlement', aliases=[], parents=['http://dbpedia.org/ontology/Settlement'], properties=[], different_froms=[], equivalent_classes=['http://schema.org/City', 'http://www.wikidata.org/entity/Q515'], ancestors=set())

In [17]:
lst[0]

RDFResource(id='http://dbpedia.org/ontology/BaseballSeason', props={'http://www.w3.org/2000/01/rdf-schema#label': [<kgdata.misc.ntriples_parser.RDFTerm object at 0x7f8562c03be0>, <kgdata.misc.ntriples_parser.RDFTerm object at 0x7f8562c02e90>, <kgdata.misc.ntriples_parser.RDFTerm object at 0x7f8562c02320>, <kgdata.misc.ntriples_parser.RDFTerm object at 0x7f8562c034c0>, <kgdata.misc.ntriples_parser.RDFTerm object at 0x7f8562c02d10>, <kgdata.misc.ntriples_parser.RDFTerm object at 0x7f8562c032b0>, <kgdata.misc.ntriples_parser.RDFTerm object at 0x7f8562c03430>], 'http://www.w3.org/2002/07/owl#sameAs': [<kgdata.misc.ntriples_parser.RDFTerm object at 0x7f8562c01d20>], 'http://www.w3.org/2007/05/powder-s#describedby': [<kgdata.misc.ntriples_parser.RDFTerm object at 0x7f8562c02860>], 'http://www.w3.org/ns/prov#wasDerivedFrom': [<kgdata.misc.ntriples_parser.RDFTerm object at 0x7f8562c01c60>], 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': [<kgdata.misc.ntriples_parser.RDFTerm object at 0x7f8

In [9]:
r = ontology_dump().get_rdd().take(1)[0]

In [15]:
r.props.get(rdf_type, [])

['http://www.w3.org/2000/01/rdf-schema#Datatype']

In [13]:
is_class(r)

False